### Bulk Ellipse Metric Calculation file for UFS Data.

File generated November 17, 2025. 

In [2]:
import datetime as dt
import numpy as np
import xarray as xr
import netCDF4
import pickle

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

import cartopy.feature
from cartopy.util import add_cyclic_point
import cartopy.crs as ccrs

#from get_ellipse_metrics import get_emetrics_max_min
from fitEllipse3_new import fitEllipseContour
from fitEllipse3_new import point_inside_polygon
#from EllipseCalc import ellipse_calc
from geopy.distance import great_circle
from matplotlib.patches import Polygon
from matplotlib import cm #colormaps!

In [3]:
from UFS_EllipseDef import UFS_one_ellipse_calc

In [ ]:
lev = [10] #Define your hPa level ... in Pa for ERA5
contour = [30000] #Define your contour relevant to hPa measures 
#20000 for 50 hPa
windlat = 60.0 #Define latitude for zonal-mean winds
#write_out = True #write the ellipse data to a text file, True or False 

In [ ]:
#running for December 15, 2011
#test = UFS_one_ellipse_calc(lev, contour, windlat, 2011, 12, 15, 36)

The final shape of this is [metric, prototype, day]. 

I can designate this to loop with "forecast year" as the first dim, "forecast number" as the second ... followed by the rest, potentially?

[forecast year,forecast number,prototype,day] ... [7 winters, 9 forecasts, 4 prototypes, 36 days]

In [ ]:
#create empty arrays for metrics
ratio = np.empty((7,8,4,36))
wind10 = np.empty((7,8,4,36))
cenlat10 = np.empty((7,8,4,36))
cenlon10 = np.empty((7,8,4,36))
size10 = np.empty((7,8,4,36))
ephi10 = np.empty((7,8,4,36))
number10 = np.empty((7,8,4,36))

In [ ]:
#lists to loop through for naming strings per season. 
years = [y for y in range(2011,2018,1)]
month_date = [(11,1),(11,15),(12,1),(12,15),(1,1),(1,15),(2,1),(2,15)]

In [ ]:
#conduct loop to calculate and save metrics. 
for i in range(len(years)):
    for j in range(len(month_date)):
        if month_date[j][0] == 11 or month_date[j][0] == 12:
            year = years[i]
            calc = UFS_one_ellipse_calc(lev, contour, windlat, year, month_date[j][0], month_date[j][1], 36)
    
            ratio[i,j] = calc[0]
            wind10[i,j] = calc[1]
            cenlat10[i,j] = calc[2]
            cenlon10[i,j] = calc[3]
            size10[i,j] = calc[4]
            ephi10[i,j] = calc[5]
            number10[i,j] = calc[6]
            
        else:
            year = years[i]+1
            calc = UFS_one_ellipse_calc(lev, contour, windlat, year, month_date[j][0], month_date[j][1], 36)
            
            ratio[i,j] = calc[0]
            wind10[i,j] = calc[1]
            cenlat10[i,j] = calc[2]
            cenlon10[i,j] = calc[3]
            size10[i,j] = calc[4]
            ephi10[i,j] = calc[5]
            number10[i,j] = calc[6]

In [ ]:
#export/dump the arrays into file
import pickle
pickle.dump(ratio, open("./UFS_metrics/UFS_ratio10.p", 'wb'))
pickle.dump(wind10, open("./UFS_metrics/UFS_wind10.p", 'wb')) 
pickle.dump(cenlat10, open("./UFS_metrics/UFS_cenlat10.p", 'wb'))
pickle.dump(cenlon10, open("./UFS_metrics/UFS_cenlon10.p", 'wb'))
pickle.dump(size10, open("./UFS_metrics/UFS_size10.p", 'wb'))
pickle.dump(ephi10, open("./UFS_metrics/UFS_ephi10.p", 'wb'))
pickle.dump(number10, open("./UFS_metrics/UFS_number10.p", 'wb'))

So, just to note. All of the metrics calculated here are for 10hPa. 

In [4]:
##import ellipse metrics from ERA5
infile = open("../extended_diags/diags/wind10_redo.p", 'rb') 
wind = pickle.load(infile)
infile.close()

infile = open("../extended_diags/diags/size10_redo.p", 'rb') 
size = pickle.load(infile)
infile.close()

infile = open("../extended_diags/diags/ratio10_redo.p", 'rb') 
rat = pickle.load(infile)
infile.close()

infile = open("../extended_diags/diags/ephi10_redo.p", 'rb') 
ephi = pickle.load(infile)
infile.close()

infile = open("../extended_diags/diags/cenlat10_redo.p", 'rb')
cenlat = pickle.load(infile)
infile.close()

infile = open("../extended_diags/diags/cenlon10_redo.p", 'rb') 
cenlon = pickle.load(infile)
infile.close()

infile = open("../extended_diags/diags/number10_redo.p", 'rb') 
number = pickle.load(infile)
infile.close()

#remove leap year
wind = np.delete(wind[:,31:],[120],1)
rat = np.delete(rat[:,31:],[120],1)
cenlat = np.delete(cenlat[:,31:],[120],1)
cenlon = np.delete(cenlon[:,31:],[120],1)
size = np.delete(size[:,31:],[120],1)
ephi = np.delete(ephi[:,31:],[120],1)
num = np.delete(number[:,31:],[120],1)

In [8]:
num[54,106]

2.0

In [13]:
#create empty arrays for ERA5 metrics
r = np.empty((7,8,36))
w = np.empty((7,8,36))
ct = np.empty((7,8,36))
cn = np.empty((7,8,36))
sz = np.empty((7,8,36))
ep = np.empty((7,8,36))
n = np.empty((7,8,36))

In [14]:
years = [y for y in range(52,59,1)]
index = [0,14,30,44,61,75,92,106]

In [15]:
for i in range(len(years)):
    year = years[i] #make sure proper index is used for the ERA-5.
    #i is preserved to index the copy array properly. 
    for j in range(len(index)):
        ind = index[j]
        r[i,j,:] = rat[year,ind:ind+36]
        w[i,j,:] = wind[year,ind:ind+36]
        ct[i,j,:] = cenlat[year,ind:ind+36]
        cn[i,j,:] = cenlon[year,ind:ind+36]
        sz[i,j,:] = size[year,ind:ind+36]
        ep[i,j,:] = ephi[year,ind:ind+36]
        n[i,j,:] = num[year,ind:ind+36]

In [18]:
#export/dump the arrays into file
pickle.dump(r, open("./UFS_metrics/ERA5_ratio10.p", 'wb'))
pickle.dump(w, open("./UFS_metrics/ERA5_wind10.p", 'wb')) 
pickle.dump(ct, open("./UFS_metrics/ERA5_cenlat10.p", 'wb'))
pickle.dump(cn, open("./UFS_metrics/ERA5_cenlon10.p", 'wb'))
pickle.dump(sz, open("./UFS_metrics/ERA5_size10.p", 'wb'))
pickle.dump(ep, open("./UFS_metrics/ERA5_ephi10.p", 'wb'))
pickle.dump(n, open("./UFS_metrics/ERA5_number10.p", 'wb'))